In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [91]:
data_train = pd.read_csv('/content/train.csv')
data_test = pd.read_csv('/content/test.csv')


In [92]:
data_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [93]:
data_train.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [145]:

def data_clean(data):
  st = StandardScaler()
  data_clean = data.copy()
  data_clean["Title"]=data_clean["Name"].str.extract(r',\s*([A-Za-z]+\.?)')
  data_clean['Age'].fillna(data_clean.groupby('Title')['Age'].transform('mean'), inplace=True)
  data_clean['Age'].dropna()
  data_clean['Fare'] =st.fit_transform(data_clean[['Fare']])
  data_clean['Fare'].fillna(data_clean.groupby('Pclass')['Fare'].transform('median'), inplace=True)
  data['Age']=st.fit_transform(data[['Age']])
  data_clean['Embarked']= data_clean['Embarked'].fillna('S')
  data_clean = data_clean.drop('Cabin', axis=1)
  data_clean = data_clean.drop('PassengerId', axis=1)
  data_clean = data_clean.drop('Name', axis=1)
  data_clean = data_clean.drop('Title',axis = 1)
  data_clean = data_clean.drop('Ticket',axis = 1)
  return data_clean



In [146]:
data_train_clean = data_clean(data_train)
data_test_clean = data_clean(data_test)

In [147]:
data_train_clean.isnull().sum()

,0
Survived,0
Pclass,0
Sex,0
Age,0
SibSp,0
Parch,0
Fare,0
Embarked,0


In [164]:
data_test_clean['Age']=data_test_clean['Age'].fillna(data_test_clean['Age'].mean())

In [165]:
data_test_clean.isnull().sum()

,0
Pclass,0
Sex,0
Age,0
SibSp,0
Parch,0
Fare,0
Embarked,0


In [166]:
data_test_clean[data_test_clean['Age'].isnull()]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked


In [168]:
en  = LabelEncoder()
data_train_clean['Sex'] = en.fit_transform(data_train_clean['Sex'])
data_train_clean['Embarked'] = en.fit_transform(data_train_clean['Embarked'])

In [169]:
data_test_clean['Sex'] = en.fit_transform(data_test_clean['Sex'])
data_test_clean['Embarked'] = en.fit_transform(data_test_clean['Embarked'])

In [170]:
data_train_clean

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,-0.530377,1,0,-0.502445,2
1,1,1,0,0.571831,1,0,0.786845,0
2,1,3,0,-0.254825,0,0,-0.488854,2
3,1,1,0,0.365167,1,0,0.420730,2
4,0,3,1,0.365167,0,0,-0.486337,2
...,...,...,...,...,...,...,...,...
886,0,2,1,-0.185937,0,0,-0.386671,2
887,1,1,0,-0.737041,0,0,-0.044381,2
888,0,3,0,-0.545947,1,2,-0.176263,2
889,1,1,1,-0.254825,0,0,-0.044381,0


In [171]:
le = LogisticRegression()
y = data_train_clean['Survived']
x = data_train_clean.drop('Survived', axis=1)
x_train , x_val , y_train , y_val = train_test_split(x,y, test_size=0.2, random_state=42)
le.fit(x_train, y_train)

LogisticRegression()

In [172]:
preditions = le.predict(x_val)

In [173]:
accuracy_score(y_val, preditions)

0.8100558659217877

In [174]:
submission_preds = le.predict(data_test_clean)

In [175]:
df = pd.DataFrame({'PassengerId': data_test['PassengerId'], 'Survived': submission_preds})

In [176]:
df.to_csv('submission.csv', index=False)